In [2]:
import torch
import tensorflow as tf
import onnx, onnx_tf
import models
import torchvision
import dill

In [3]:
TORCH_WEIGHTS = 'torch_weights/02-Resnet50_epoch-0028_acc-0.614.pth'
ONNX_EXPORT_PATH = 'onnx_models/02-Resnet50_epoch-0028_acc-0.614.onnx'
TF_EXPORT_PATH = 'tf_models/02-Resnet50_epoch-0028_acc-0.614.pb'

In [4]:
# Create directories for output
%mkdir onnx_models
%mkdir tf_models

mkdir: cannot create directory ‘onnx_models’: File exists
mkdir: cannot create directory ‘tf_models’: File exists


In [5]:
torch_model = models.Resnet(torchvision.models.resnet50, 2)

In [6]:
torch_model.load_state_dict(torch.load(TORCH_WEIGHTS, pickle_module=dill, map_location='cpu'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [7]:
dummy_input = torch.rand(1, 3, 224, 224)

In [8]:
dummy_output = torch_model(dummy_input)

In [9]:
dummy_output

tensor([[ 0.3208, -0.3254]], grad_fn=<AddmmBackward>)

In [10]:
torch.onnx.export(torch_model, dummy_input, ONNX_EXPORT_PATH, input_names=['input'], 
                  output_names=['output'])

In [11]:
onnx_model = onnx.load(ONNX_EXPORT_PATH)

In [12]:
tf_rep = onnx_tf.backend.prepare(onnx_model)

/usr/local/lib/python3.6/site-packages/onnx_tf/common/handler_helper.py:37: UserWarning: Unknown op ConstantFill in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/usr/local/lib/python3.6/site-packages/onnx_tf/common/handler_helper.py:37: UserWarning: Unknown op ImageScaler in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/usr/local/lib/python3.6/site-packages/onnx_tf/common/handler_helper.py:34: UserWarning: Fail to get since_version of IsInf in domain `` with max_inclusive_version=9. Set to 1.
  handler.ONNX_OP, handler.DOMAIN, version))
/usr/local/lib/python3.6/site-packages/onnx_tf/common/handler_helper.py:34: UserWarning: Fail to get since_version of Mod in domain `` with max_inclusive_version=9. Set to 1.
  handler.ONNX_OP, handler.DOMAIN, version))
/usr/local/lib/python3.6/site-packages/onnx_tf/common/handler_helper.py:34: UserWarning: Fail to get since_version of ThresholdedRelu in domain `` with max_inclusive_version=9. Set to 1.
  hand

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


/usr/local/lib/python3.6/site-packages/onnx_tf/handlers/backend/pool_mixin.py:124: UserWarning: Using the pooling op in compatibility mode.This means your graph cannot be serialized.Please configure your pooling operation to only use paddings that correspond to Tensorflow SAME or VALID padding.
  "correspond to Tensorflow SAME or VALID padding.", UserWarning)


Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Instructions for updating:
Use keras.layers.flatten instead.


In [13]:
tf_rep.tensor_dict

{'backbone.0.weight': <tf.Tensor 'Const:0' shape=(64, 3, 7, 7) dtype=float32>,
 'backbone.1.bias': <tf.Tensor 'Const_1:0' shape=(64,) dtype=float32>,
 'backbone.1.num_batches_tracked': <tf.Tensor 'Const_2:0' shape=() dtype=int64>,
 'backbone.1.running_mean': <tf.Tensor 'Const_3:0' shape=(64,) dtype=float32>,
 'backbone.1.running_var': <tf.Tensor 'Const_4:0' shape=(64,) dtype=float32>,
 'backbone.1.weight': <tf.Tensor 'Const_5:0' shape=(64,) dtype=float32>,
 'backbone.4.0.bn1.bias': <tf.Tensor 'Const_6:0' shape=(64,) dtype=float32>,
 'backbone.4.0.bn1.num_batches_tracked': <tf.Tensor 'Const_7:0' shape=() dtype=int64>,
 'backbone.4.0.bn1.running_mean': <tf.Tensor 'Const_8:0' shape=(64,) dtype=float32>,
 'backbone.4.0.bn1.running_var': <tf.Tensor 'Const_9:0' shape=(64,) dtype=float32>,
 'backbone.4.0.bn1.weight': <tf.Tensor 'Const_10:0' shape=(64,) dtype=float32>,
 'backbone.4.0.bn2.bias': <tf.Tensor 'Const_11:0' shape=(64,) dtype=float32>,
 'backbone.4.0.bn2.num_batches_tracked': <tf.Ten

In [14]:
tf_rep.export_graph(TF_EXPORT_PATH)

In [15]:
def load_pb(path_to_pb):
    with tf.gfile.GFile(path_to_pb, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name='')
        return graph

In [16]:
graph = load_pb(TF_EXPORT_PATH)

In [17]:
sess = tf.Session(graph=graph)

In [18]:
output_tensor = graph.get_tensor_by_name('add_18:0')
input_tensor = graph.get_tensor_by_name('input:0')

In [19]:
output = sess.run(output_tensor, feed_dict={input_tensor: dummy_input.numpy()})

In [20]:
output

array([[-0.48209134,  0.4726165 ]], dtype=float32)